In [ ]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.8/906.8 kB 15.0 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 72.7MB/s]


In [ ]:


import json
from ultralytics import YOLO

model = YOLO('yolov8n.pt')


results = model('/content/Traffic video  for object detection and tracking.mp4')


json_data = []


for result in results:
    boxes = result.boxes
    frame_data = {}
    detections = []


    for box in boxes:
        detection = {
            'coordinates': box.xyxy[0].tolist(),
            'confidence': float(box.conf[0]),
            'class': model.names[int(box.cls[0])]
        }

        detections.append(detection)

    frame_data["detections"] = detections
    json_data.append(frame_data)


json_string = json.dumps(json_data, indent=4)


print(json_string)
with open('detections.json', 'w') as f:
    f.write(json_string)

Streaming output truncated to the last 5000 lines.
                    206.03890991210938,
                    122.68335723876953
                ],
                "confidence": 0.45122069120407104,
                "class": "car"
            },
            {
                "coordinates": [
                    283.62567138671875,
                    101.62085723876953,
                    298.137939453125,
                    116.55286407470703
                ],
                "confidence": 0.4408716559410095,
                "class": "car"
            }
        ]
    },
    {
        "detections": [
            {
                "coordinates": [
                    597.1533813476562,
                    147.43087768554688,
                    639.7092895507812,
                    188.61715698242188
                ],
                "confidence": 0.5905708074569702,
                "class": "car"
            },
            {
                "coordinates": [
                    182

In [ ]:
# prompt: so i need the output in video where it showing the results of the detecting the objects in the video

from ultralytics import YOLO
import json

# Load a model
model = YOLO('yolov8n.pt')

# Run inference on a video
results = model('/content/Traffic video  for object detection and tracking.mp4', save=True, project='yolov8_detections')





WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/905) /content/Traffic video  for object detection and tracking.mp4: 352x640 12 cars, 1 truck, 129.9ms
video 1/1 (frame 2/905) /content/Traffic video  for object detection and tracking.mp4: 352x640 11 cars, 120.4ms
video 1/1 (frame 3/905) /content/Traffic video  for object detection and tracking.mp4: 352x640 12 cars, 117.1ms
video 1/1 (frame 4/905) /content/Traffic video  for object detection and tracking.mp4: 352x640 12 cars, 121.3ms

In [ ]:

json_data = []


for result in results:
    boxes = result.boxes
    frame_data = {}
    detections = []


    for i, box in enumerate(boxes):
        detection = {
            "object": model.names[int(box.cls[0])],
            "id": i,
            "bbox": box.xyxy[0].tolist(),
        }

        if i % 2 == 0 and len(boxes) > i +1:
          detection["subobject"] = {
              "object": model.names[int(boxes[i+1].cls[0])],
              "id": i+1,
              "bbox": boxes[i+1].xyxy[0].tolist()
          }

        detections.append(detection)

    frame_data["detections"] = detections
    json_data.append(frame_data)


json_string = json.dumps(json_data, indent=4)

print(json_string)

with open('detections_with_subobjects.json', 'w') as f:
    f.write(json_string)

[
    {
        "detections": [
            {
                "object": "car",
                "id": 0,
                "bbox": [
                    230.86514282226562,
                    129.59483337402344,
                    289.5321960449219,
                    172.60206604003906
                ],
                "subobject": {
                    "object": "car",
                    "id": 1,
                    "bbox": [
                        347.3335266113281,
                        132.1451416015625,
                        414.3849792480469,
                        165.33526611328125
                    ]
                }
            },
            {
                "object": "car",
                "id": 1,
                "bbox": [
                    347.3335266113281,
                    132.1451416015625,
                    414.3849792480469,
                    165.33526611328125
                ]
            },
            {
                "object": "car",
     

In [ ]:
# prompt: 3. Sub-Object Image Retrieval
#  ● Add functionality to retrieve and save cropped images of specific sub-objects (e.g.,
#  the helmet of "Person A" or Tire 1 of Car 1 etc).
#  ● Ensure the retrieval system uses the hierarchical association between objects and
#  sub-objects for accurate image extraction.

from PIL import Image
import json


def crop_and_save_subobjects(json_data, image_path, output_dir='cropped_images'):

    import os
    os.makedirs(output_dir, exist_ok=True)

    for frame_data in json_data:
        img = Image.open(image_path)
        for detection in frame_data['detections']:
            if 'subobject' in detection:
                subobject = detection['subobject']
                bbox = subobject['bbox']


                cropped_img = img.crop(bbox)


                filename = f"{frame_data.get('frame_id', 0)}_{detection['id']}_{subobject['id']}_{subobject['object']}.jpg"
                output_path = os.path.join(output_dir, filename)

                cropped_img.save(output_path)


import cv2
import os

video_path = '/content/Traffic video  for object detection and tracking.mp4'
output_dir = "cropped_images"
os.makedirs(output_dir, exist_ok=True)
vidcap = cv2.VideoCapture(video_path)
success,image = vidcap.read()
count = 0
success = True
while success:
  success,image = vidcap.read()
  if not success:
    break
  cv2.imwrite(f"/content/frame{count}.jpg", image)     # save frame as JPEG file
  crop_and_save_subobjects(json_data, f"/content/frame{count}.jpg", output_dir)
  count += 1

In [ ]:
# prompt: 4. Inference Speed Optimization
# ● Optimize the system to process video input with an inference speed of 10–30 frames
#  per second (FPS) on CPU.
#  ● Benchmark the inference speed on CPU using a sample video and document the
#  results.

import time

# Load a model
model = YOLO('yolov8n.pt')

# Path to your video file
video_path = '/content/Traffic video  for object detection and tracking.mp4'

# Open the video file
cap = cv2.VideoCapture(video_path)

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v') # You might need to change this codec
out = cv2.VideoWriter('output.mp4', fourcc, fps, (frame_width, frame_height))

# Initialize variables for FPS calculation
start_time = time.time()
frame_count = 0

while cap.isOpened():
    success, frame = cap.read()

    if success:
        # Run YOLOv8 inference
        results = model(frame)

        # Annotate the frame with bounding boxes (optional)
        annotated_frame = results[0].plot()

        # Write the frame to the output video
        out.write(annotated_frame)

        frame_count += 1

        # Calculate FPS every 10 frames
        if frame_count % 10 == 0:
          end_time = time.time()
          elapsed_time = end_time - start_time
          fps = frame_count / elapsed_time
          print(f"Current FPS: {fps:.2f}")
          start_time = end_time
          frame_count = 0


    else:
        break

# Release resources
cap.release()
out.release()
print("Video processing complete.")


In [ ]:
# prompt:  5. Modularity and Extensibility
#  ● Ensure the system design is modular, allowing for easy addition of new
#  object-sub-object pairs (e.g., cars and license plates).
#  ● Provide clear instructions for adapting the system to other object detection scenarios.

import json
import os
from PIL import Image

def process_detections(results, model):


    json_data = []
    for result in results:
        boxes = result.boxes
        frame_data = {}
        detections = []

        for i, box in enumerate(boxes):
            detection = {
                "object": model.names[int(box.cls[0])],
                "id": i,
                "bbox": box.xyxy[0].tolist(),
            }


            if model.names[int(box.cls[0])] == "car" and i % 2 == 0 and len(boxes) > i+1:
                  "object": "license_plate",
                  "id": i+1,
                  "bbox": boxes[i+1].xyxy[0].tolist()
              }
            detections.append(detection)

        frame_data["detections"] = detections
        json_data.append(frame_data)

    return json_data



def add_new_object_subobject_pair(model, object_name, subobject_name):
  """
  Adds a new object-subobject pair to the model.
  This is a placeholder function. In a real system,
  you would need to update the model or its config file
  to include the new object and subobject. You might also need
  to retrain the model with data of the new object and its subobjects.

  Args:
    model: The YOLO model
    object_name (str): Name of the new object.
    subobject_name (str): Name of the new subobject.
  """
  print(f"Simulating adding new object-subobject pair: {object_name} - {subobject_name}")

add_new_object_subobject_pair(model, "bicycle", "wheel")



Simulating adding new object-subobject pair: bicycle - wheel
